In [7]:
import json
import os
import tqdm
parent_dir = "../dataset/"
describe_filename_list = []
for item in os.listdir(parent_dir):
    if item.startswith("instruct_uie_ner_converted_description_"):
        describe_filename_list.append(item)
data = []
for item in tqdm.tqdm( describe_filename_list):
    file_path = os.path.join(parent_dir, item)
    with open(file_path,"r", encoding="utf-8") as f:
        content = f.readlines()
    for line in content:
        obj = json.loads(line.strip())
        data.append(obj)

In [4]:
# 把相关的jsonl数据整合在一起

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 412/412 [00:01<00:00, 374.70it/s]


# task 1 

In [11]:
task_1_template = """You are given a text and a set of target labels.

Generate descriptions ONLY for the labels listed below.
Do NOT generate any labels that are not listed.

<Text>
{{TEXT}}
</Text>

<LABEL_SET>
{{LABEL_1}}
{{LABEL_2}}
{{LABEL_3}}
...
</LABEL_SET>

Output format (STRICT):
Each line must be:
LABEL: DESCRIPTION

Each LABEL must appear exactly once.
Do not add extra text or blank lines."""

# task 2 open

In [12]:
task_2_template = """
You are given a text.

Identify the labels that are relevant to this text,
and generate a description for each identified label.

<Text>
{{TEXT}}
</Text>

Output format:
Each line must be:
LABEL: DESCRIPTION

Only include labels clearly supported by the text.
Do not include speculative or irrelevant labels.
"""

# task 3

# 尝试构造gliner 数据

In [14]:
from transformers import AutoTokenizer

In [22]:
from typing import Optional, Tuple, List

def char_to_token_span(
    offsets: List[Tuple[int, int]],
    char_start: int,
    char_end: int,
    *,
    special_offset=(0, 0),
) -> Optional[Tuple[int, int]]:
    """
    Map [char_start, char_end) to token span [tok_start, tok_end] (inclusive).
    Returns None if cannot be aligned (e.g., fully truncated or only hits special tokens).
    """
    # 1) 过滤 special tokens：offset=(0,0) 或 (-1,-1) 这类
    valid = []
    for i, (s, e) in enumerate(offsets):
        if (s, e) == special_offset or (s == 0 and e == 0) or (s < 0 and e < 0):
            continue
        valid.append((i, s, e))
    if not valid:
        return None

    # 2) 找 tok_start：第一个满足 token_end > char_start 的 token
    tok_start = None
    for i, s, e in valid:
        if e > char_start:
            tok_start = i
            break

    # 3) 找 tok_end：最后一个满足 token_start < char_end 的 token
    tok_end = None
    for i, s, e in reversed(valid):
        if s < char_end:
            tok_end = i
            break

    if tok_start is None or tok_end is None or tok_start > tok_end:
        return None

    # 4) 可选：严格边界检查（防止映射到完全不相交的 token）
    # 要求实体与 tok_start/tok_end 的 token 区间至少有交集
    s0, e0 = offsets[tok_start]
    s1, e1 = offsets[tok_end]
    if not (e0 > char_start and s1 < char_end):
        return None

    return tok_start, tok_end

In [15]:
tokenizer  = AutoTokenizer.from_pretrained("/data/model_hub/mdeberta-v3-base/")

In [18]:
text = data[0]["sentence"]

In [21]:
enc = tokenizer(
    text,
    return_offsets_mapping=True,
    truncation=True,
    max_length=512,
    return_tensors=None,
)
offsets = enc["offset_mapping"]  # list[(char_s, char_e)] per token

In [37]:
pos = data[0]["entities"][1]["pos"]

In [36]:
data[0]["entities"][1]["name"]

'activity_horizontal_margin'

In [38]:
text[pos[0]:pos[1]]

'activity_horizontal_margin'

In [41]:
offset_pos = char_to_token_span(offsets, pos[0],pos[1])

In [42]:
tokenizer.convert_ids_to_tokens(enc["input_ids"][offset_pos[0]:offset_pos[1]+1])

['activity', '_', 'horizontal', '_', 'margin']

In [43]:
data[0]

{'sentence': '<resources>\n    <!-- Example customization of dimensions originally defined in res/values/dimens.xml\n         (such as screen margins) for screens with more than 820dp of available width. This\n         would include 7" and 10" devices in landscape (~960dp and ~1280dp respectively). -->\n    <dimen name="activity_horizontal_margin">64dp</dimen>\n</resources>',
 'entities': [{'name': '64dp', 'pos': [332, 336], 'type': 'dimension value'},
  {'name': 'activity_horizontal_margin',
   'pos': [304, 330],
   'type': 'dimension name'},
  {'name': 'res/values/dimens.xml', 'pos': [79, 100], 'type': 'file path'}],
 'dataset': 'Pile_NER_type',
 'types': ['dimension name', 'dimension value', 'file path'],
 'description': [{'label': 'dimension value',
   'definitions': {'D1': "A string appearing as the content of a dimen element, following the opening tag and preceding the closing tag, formatted with a numeric value followed by a unit suffix like 'dp'.",
    'D2': "The text inside a 